# Auto Generated Agent Chat: Multi Task Chats

This notebook showcases how to use the new chat interface of conversational agents in AutoGen: initiate_chats, to conduct a series of tasks. This new interface allows one to pass multiple tasks and their corresponding dedicated agents. Once initiate_chats is invoked, the tasks will be solved sequentially, with the summaries from previous tasks provided to subsequent tasks as context, if the `summary_method` argument is specified.


## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

### Example Tasks
Below are three example tasks, with each task being a string of text describing the request. The completion of later tasks requires or benefits from the results of previous tasks.


In [1]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list}

financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Scenario 1: Solve the tasks with a series of chats

The `initiate_chats` interface can take a list of dictionaries as inputs. Each dictionary preserves the following fields: 
- `message`: is a string of text (typically a message containing the task);
- `recipient`: a conversable agent dedicated for the task;
- `summary_method`: A string specifying the method to get a summary from the chat. Currently supported choices include `last_msg`, which takes the last message from the chat history as the summary, and `reflection_with_llm`, which uses an LLM call to reflect on the chat history and summarize a takeaway;
- `summary_prompt`: A string specifying how to instruct an LLM-backed agent (either the recipient or the sender in the chat) to reflect on the chat history and derive a summary. If not otherwise specified, a default prompt will be used when `summary_method` is `reflection_with_llm`.
"Summarize the takeaway from the conversation. Do not add any introductory phrases. If the intended request is NOT properly addressed, please point it out."
- `carryover`: A string or a list of string to specify additional context to be used in the chat. With `initiate_chats`, summary from previous chats will be added as carryover. They will be appended after the carryover provided by the user.

In [3]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

chat_results = user.initiate_chats(
    [
        {
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
            "carryover": "I want to include a figure or a table of data in the blogpost.",
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

With the following carryover: 


********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), along with their performance over the past month in terms of percentage change, we can use a Python library like `yfinance`. This library allows downloading of historical market data from Yahoo Finance.

Let's proceed in the following steps:

1. I will provide you

#### Check chat results
The `initiate_chat` method returns a `ChatResult` object, which is a dataclass object storing information about the chat. Currently, it includes the following attributes:

- `chat_history`: a list of chat history.
- `summary`: a string of chat summary. A summary is only available if a summary_method is provided when initiating the chat.
- `cost`: a tuple of (total_cost, total_actual_cost), where total_cost is a dictionary of cost information, and total_actual_cost is a dictionary of information on the actual incurred cost with cache.
- `human_input`: a list of strings of human inputs solicited during the chat. (Note that since we are setting `human_input_mode` to `NEVER` in this notebook, this list is always empty.)

In [4]:
for agent, chat_res in chat_results.items():
    print(f"Conversation summary with {agent.name}:")
    print(chat_res.summary)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

Conversation summary with Financial_assistant:
The given Python script successfully fetched the stock prices and percentage changes over the past month for NVIDIA Corporation (NVDA) and Tesla, Inc. (TSLA). NVDA had a significant price increase of approximately 30.09%, while TSLA experienced a notable decline of about 21.85%.
Human input in the middle: []
Conversation cost:  ({'total_cost': 0.12056999999999998, 'gpt-4': {'cost': 0.12056999999999998, 'prompt_tokens': 2485, 'completion_tokens': 767, 'total_tokens': 3252}}, {'total_cost': 0.07446, 'gpt-4': {'cost': 0.07446, 'prompt_tokens': 1984, 'completion_tokens': 249, 'total_tokens': 2233}})



Conversation summary with Researcher:
NVDA is experiencing positive stock performance due to analyst upgrades, bullish sentiment on AI initiatives and growth, and strong earnings expectations. TSLA, on the other hand, is facing challenges, including regulatory scrutiny, potential recalls, concerns over financial performance and growth forecasts,

### Scenario 2: With human inputs revising tasks in the middle

Since AutoGen agents support soliciting human inputs during a chat if `human_input_mode` is specificed properly, the actual task might be revised in the middle of a chat. 

The example below showcases that even if a task is revised in the middle (for the first task, the human user requests to get Microsoft's stock price information as well, in addition to NVDA and TSLA), the `reflection_with_llm`` summary method can still capture it, as it reflects on the whole conversation instead of just the original request.

In [3]:
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",  # ask human for input at each step
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

chat_results = user.initiate_chats(
    [
        {
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

With the following carryover: 


********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), along with their performance over the past month in terms of percentage change, we can use a Python library like `yfinance`. This library allows downloading of historical market data from Yahoo Finance.

Let's proceed in the following steps:

1. I will provide you

/Users/qingyunwu/Documents/github/autogen/autogen/agentchat/conversable_agent.py:798: UserWarning: No summary_method provided or summary_method is not supported: 
  warnings.warn("No summary_method provided or summary_method is not supported: ")


#### Check chat results

In [5]:
for agent, chat_res in chat_results.items():
    print(f"Conversation summary with {agent.name}:")
    print(chat_res.summary)
    print("All chat history:", chat_res.chat_history)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

Conversation summary with Financial_assistant:
The current stock prices and the past month's performance in percentage change for NVIDIA Corporation (NVDA), Tesla, Inc. (TSLA), and Microsoft Corporation (MSFT) were as follows:

- NVDA: $677.43 with a 29.64% increase.
- TSLA: $176.10 with a 26.76% decrease.
- MSFT: $404.79 with an 8.03% increase.

The stock prices reflect the values at the time the Python script using the `yfinance` library was executed, and the performance is based on the past 30 days from that date. Stock prices are subject to frequent fluctuations during trading hours.
All chat history: [{'content': 'What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?', 'role': 'assistant'}, {'content': 'To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), along with their performance over the past month in terms of percentage change, we can use a Python library like `yfinan

### Scenario 3: Solve the tasks with a series of chats involving group chat

In [12]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    Reply "TERMINATE" in the end when everything is done.
    """,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.
    Reply "TERMINATE" in the end when everything is done.
    """,
    llm_config={"config_list": config_list},
)

groupchat_1 = autogen.GroupChat(agents=[user_proxy, research_assistant, critic], messages=[], max_round=50)

groupchat_2 = autogen.GroupChat(agents=[user_proxy, writer, critic], messages=[], max_round=50)

manager_1 = autogen.GroupChatManager(
    groupchat=groupchat_1,
    name="Research_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)
manager_2 = autogen.GroupChatManager(
    groupchat=groupchat_2,
    name="Writing_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
user.initiate_chats(
    [
        {"recipient": research_assistant, "message": financial_tasks[0], "summary_method": "last_msg"},
        {"recipient": manager_1, "message": financial_tasks[1], "summary_method": "reflection_with_llm"},
        {"recipient": manager_2, "message": writing_tasks[0]},
    ]
)


********************************************************************************
Start a new chat with the following message: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

With the following carryover: 


********************************************************************************
User (to Researcher):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Researcher (to User):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), along with their performance over the past month in terms of percentage change, we can use a Python library like `yfinance`. This library allows downloading of historical market data from Yahoo Finance.

Let's proceed in the following steps:

1. I will provide you a Python script t